<a href="https://colab.research.google.com/github/ckj18/ToBigs_workspace/blob/main/Week1_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

라이브러리는 일종의 도구로 개발의 툴인 프레임워크와 달리 제약없이 자유롭게 사용이 가능하지만, 프레임워크는 메뉴얼에 따라 프로그램을 만든다.

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

딥러닝은 머신러닝에 속해 있으며, 머신러닝은 컴퓨터가 스스로 학습하여 AI 성능을 향상시키는 기술이고 딥러닝은 인간의 뉴런과 비슷한 인공신경망 방식으로 정보를 처리하는 것이다. 

### Q3. 아래의 코드에 주석 달기.

In [ ]:
## 한 GPU에만 Cuda 연산을 할당하기 위해 사용 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

In [ ]:
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [ ]:
## MNIST 데이터 셋에서 train set과 test set을 지정하고 transformer를 통해 텐서로 변환
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

In [ ]:
## batch_size는 100, 데이터 셋에서 batch를 통해 100개씩 묶어 파라미터를 업데이트 
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

In [ ]:
## 학습에 사용되는 모델로, 사용되는 layer function은 init에 정의되어 있으며, forward를 통해 layer 층을 구성
## Conv -> MaxPool -> ReLu -> Conv -> MaxPool -> ReLu -> DropOut -> Flatten -> fc -> fc -> log_softmax의 순서로 진행
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) 
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 


  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
## ConvNet 모델과 CrossEntropyLoss를 사용하고 Optimizer로 Adam, lr=0.001로 지정
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
## 5번의 반복 학습을 하며, 모델을 통해 학습한 예측값 hypothesis와 정답 label인 target 값의 loss를 구하고 최적화를 통해 진행
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward()
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:
## test에는 no_grad()를 사용해야 하며, train에서 학습된 모델로, 예측값 preds와 정답 label target 값이 일치하는 correct의 비율을 계산 
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.